In [ ]:
# import the proper packages
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

## Gourmet Gizmos

Assume that 3 plant locations are up and running so you don't have to worry about fixed costs.

In [ ]:
# Start with Gourmet Gizmos
# Get data
# Gourmet Gizmos demand is in a sheet called 'gg_demand'


In [ ]:
# Convert demand to a dictionary


In [ ]:
# Create the markets as a list


In [ ]:
# Get data
# the capacity, variable, and fixed costs are in a sheet labeled 'gg_costs_and_cap'


In [ ]:
# Create the plants as a list


In [ ]:
# Create a dictionary to hold the capacity for each plant


In [ ]:
# Create a dictionary to hold the fixed cost for each plant


In [ ]:
# Get data
# Transportation costs are in our gg_costs_and_cap DataFrame


In [ ]:
# Convert costs to a list of lists


In [ ]:
# Create the model instance and set to minimize


In [ ]:
# Create the flow variables from plants to markets


# Update the model and display


In [ ]:
# Make sure you meet demand for each market


# Update and display


In [ ]:
# Make sure stay under copacity at plants


# Update and display


In [ ]:
# Solve


In [ ]:
# Print answer


## Culinary Compacts

Assume the two supply locations are already up and running so you do not need to incorporate the fixed costs.

In [ ]:
# Culinary Compacts
# Get data
# Culinary Compacts demand is in a sheet called 'cc_demand'


In [ ]:
# Convert demand to a dictionary


In [ ]:
# Create the markets as a list


In [ ]:
# Get data
# the capacity, variable, and fixed costs are in a sheet labeled 'gg_costs_and_cap'


In [ ]:
# Create the plants as a list


In [ ]:
# Create a dictionary to hold the capacity for each plant


In [ ]:
# Create a dictionary to hold the fixed cost for each plant


In [ ]:
# Get data
# Transportation costs are in our cc_costs_and_cap DataFrame


In [ ]:
# Convert costs to a list of lists


In [ ]:
# Create the model instance and set to minimize


# Create the flow variables from plants to markets


# Make sure you meet demand for each market


# Make sure stay under copacity at plants


# Update and display


In [ ]:
# solve


In [ ]:
# Print answer


## Combined Company - Culinary Gizmos

Now we need to incorporate the fixed costs for each manufacturing plant when deciding the best design configuration.

In [ ]:
# Culinary Gizmos - the combined company
# Get data
# Culinary Gizmos demand is in a sheet called 'combined_demand'


# Convert demand to a dictionary


# Create the markets as a list


In [ ]:
# Get data
# the capacity, variable, and fixed costs are in a sheet labeled 'combined_costs_and_cap'


In [ ]:
# Create the plants as a list


# Create a dictionary to hold the capacity for each plant


# Create a dictionary to hold the fixed cost for each plant


# Get data
# Transportation costs are in our all_costs_and_cap DataFrame


# Convert costs to a list of lists


In [ ]:
# Create the model instance and set to minimize


# Create the flow variables from plants to markets


# Create the binary variables to open plants


# update


# Make sure you meet demand for each market


# Make sure stay under copacity at plants


# Update and display


In [ ]:
# solve


In [ ]:
# Print answer
